In [11]:
#----------------------START-------------------------------
import requests
from time import time, sleep

API_URL = "http://localhost:3003/api/"
SIGNIN_ENDPOINT = "auth/signin"
DATASETS_ENDPOINT = "dataset/"

class TokenManager:
    def __init__(self, signin_data, signin_headers):
        self.signin_url = f"{API_URL}{SIGNIN_ENDPOINT}"
        self.signin_data = signin_data
        self.signin_headers = signin_headers
        self.token = None
        self.token_expiry = 0

    def get_token(self):
        """
        Obtains the authentication token. If the token is expired, it will refresh it.
        """
        if self.token is None or time() >= self.token_expiry:
            self.refresh_token()
        return self.token

    def refresh_token(self):
        """
        Performs a sign-in request and updates the token and its expiration time.
        """
        try:
            response = requests.post(self.signin_url, headers=self.signin_headers, json=self.signin_data)
            response.raise_for_status()  # Raise an exception if the request was not successful
            user_info = response.json()
            self.token = user_info.get("token")
            self.token_expiry = time() + 3600  # Assuming the token expires in 1 hour
        except requests.RequestException as e:
            print(f"Sign-in request failed: {e}")
            self.token = None
            self.token_expiry = 0

# Example usage
signin_data = {"username": "test", "password": "test"}
signin_headers = {
    "accept": "application/json",
    "content-type": "application/json"
}

token_manager = TokenManager(signin_data, signin_headers)

# Obtain the token
token = token_manager.get_token()
#print("Authentication token:", token)

In [12]:
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Now, use the detected encoding to read the file with pandas
# Replace 'detected_encoding' with the actual encoding detected by chardet
df = pd.read_csv("JOT sample original.csv", sep='\t', encoding='ISO-8859-1')


In [13]:
# Assuming df is your DataFrame
df['Fecha_id'] = pd.to_datetime(df['Fecha_id'], format='%Y%m%d')
df['Fecha_id'] = df['Fecha_id'].dt.strftime('%Y-%m-%d')

In [4]:

import zipfile
import os

# Define the path for the CSV and zip files
csv_file = 'JOT_Date_Transformed2.csv'  # Relative path
zip_file = 'JOT_Date_Transformed2.zip'  # Relative path

# Save the DataFrame to a CSV file
df.to_csv(csv_file, index=False)

# Create a zip file containing the CSV
with zipfile.ZipFile(zip_file, 'w', zipfile.ZIP_DEFLATED) as z:
    z.write(csv_file, os.path.basename(csv_file))

# Clean up: remove the CSV file if you don't need it anymore
#os.remove(csv_file)

print("Zip file created successfully.")


Zip file created successfully.


In [14]:
def get_database_list(api_url, datasets_endpoint, token_manager):
    """
    Retrieves the list of datasets from the API.

    Args:
        api_url (str): The base URL of the API.
        datasets_endpoint (str): The endpoint for retrieving the dataset list.
        token_manager (object): An object that manages the authentication token.

    Returns:
        dict: The JSON response containing the dataset information.
    """
    try:
        url = f"{api_url}{datasets_endpoint}"
        headers = {
            "accept": "application/json",
            "authorization": f"Bearer {token_manager.get_token()}"
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"Request failed: {e}")
        return {}
database_list = get_database_list(API_URL, DATASETS_ENDPOINT, token_manager)
database_list

{'meta': {'name': {'label': 'Name',
   'type': 'link',
   'props': {'url': '/datasets/:id/tables'}},
  'description': {'label': 'Description'},
  'nTables': {'label': 'N. Tables'},
  'mentions': {'label': 'N. Mentions'},
  'lastModifiedDate': {'label': 'Last Modified', 'type': 'date'}},
 'collection': [{'id': '0',
   'userId': 0,
   'name': 'Museums',
   'nTables': 6,
   'lastModifiedDate': '2023-11-06T10:34:36.196Z'},
  {'id': '1',
   'userId': 0,
   'name': 'JOT BC',
   'nTables': 7,
   'lastModifiedDate': '2023-11-06T13:11:29.481Z'},
  {'id': '2',
   'userId': 0,
   'name': 'SN BC',
   'nTables': 8,
   'lastModifiedDate': '2023-11-15T09:51:35.102Z'},
  {'id': '3',
   'userId': 0,
   'name': 'InterTwino',
   'nTables': 7,
   'lastModifiedDate': '2023-12-15T13:31:24.769Z'},
  {'id': '4',
   'userId': 0,
   'name': 'New Dataset',
   'nTables': 1,
   'lastModifiedDate': '2024-04-26T09:04:24.453Z'},
  {'id': '5',
   'userId': 0,
   'name': 'JOT Dataset',
   'nTables': 1,
   'lastModified

In [6]:
import requests
import json
import io

# The endpoint URL for adding a dataset
API_URL = "http://localhost:3003/api/"
DATASETS_ENDPOINT = "dataset"
url = f"{API_URL}{DATASETS_ENDPOINT}"

token = token_manager.get_token()

# Set the headers with the token
headers = {
    'Authorization': f'Bearer {token}'
}

# Path to the zip file you want to upload
zip_file_path = 'JOT_Date_Transformed2.zip'

# Open the zip file in binary mode
with open(zip_file_path, 'rb') as file:
    files = {
        'file': (file.name, file, 'application/zip')
    }

    data = {
        'name': 'JOT_May2'
    }

    # Send the POST request to add the dataset
    response = requests.post(url, headers=headers, data=data, files=files, timeout=30)

# Check the response
if response.status_code == 200:
    print("Dataset added successfully!")
    # Extract the dataset ID from the response
    response_data = response.json()
    dataset_id = response_data['datasets'][0]['id']
    print(f"New dataset ID: {dataset_id}")
elif response.status_code == 400:
    # Extract the error message and dataset ID from the response
    response_data = response.json()
    error_message = response_data.get('error', 'Unknown error')
    dataset_id = response_data.get('datasetId')
    print(f"Error: {error_message}")
    print(f"Dataset ID: {dataset_id}")
else:
    print(f"Failed to add dataset: {response.status_code}, {response.text}")

Dataset added successfully!
New dataset ID: 13


In [15]:
import requests
import pandas as pd
import json

API_URL = "http://localhost:3003/api/"
DATASETS_ENDPOINT = "dataset/"

def get_dataset_tables(dataset_id, token_manager):
    """
    Retrieves the list of tables for a given dataset.
    
    Args:
        dataset_id (str): The ID of the dataset.
        token_manager (TokenManager): An instance of the TokenManager class.
    
    Returns:
        list: A list of tables in the dataset.
    """
    try:
        url = f"{API_URL}{DATASETS_ENDPOINT}{dataset_id}/table"
        headers = {
            "accept": "application/json",
            "authorization": f"Bearer {token_manager.get_token()}"
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an exception if the request was not successful
        return response.json()["collection"]
    except (requests.RequestException, json.JSONDecodeError, KeyError) as e:
        print(f"Error getting dataset tables: {e}")
        return []


def get_table(dataset_id, table_id, token_manager):
    """
    Retrieves a table by its ID from a specific dataset.
    
    Args:
        dataset_id (str): The ID of the dataset.
        table_id (str): The ID of the table to retrieve.
        token_manager (TokenManager): An instance of the TokenManager class.
    
    Returns:
        dict: The table data in JSON format.
    """
    url = f"{API_URL}{DATASETS_ENDPOINT}{dataset_id}/table/{table_id}"
    headers = {
        "Authorization": f"Bearer {token_manager.get_token()}",
        "Accept": "application/json"
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an exception if the request was not successful
        table_data = response.json()
        
        return table_data
    
    except requests.RequestException as e:
        print(f"Error occurred while retrieving the table data: {e}")
        return None

def get_table_by_name(dataset_id, table_name, token_manager):
    """
    Retrieves a table by its name from a specific dataset.
    
    Args:
        dataset_id (str): The ID of the dataset.
        table_name (str): The name of the table to retrieve.
        token_manager (TokenManager): An instance of the TokenManager class.
    
    Returns:
        dict: The table data in JSON format.
    """
    tables = get_dataset_tables(dataset_id, token_manager)
    
    for table in tables:
        if table["name"] == table_name:
            table_id = table["id"]
            return get_table(dataset_id, table_id, token_manager)
    
    print(f"Table '{table_name}' not found in the dataset.")
    return None

# Example usage
dataset_id = "13"
table_name = "JOT_Date_Transformed2"  # Replace with the desired table name

table_json = get_table_by_name(dataset_id, table_name, token_manager)

if table_json is not None:
    # Process the table data as needed
    print("Table JSON:")
    print(json.dumps(table_json, indent=2))

Table JSON:
{
  "table": {
    "id": "44",
    "idDataset": "13",
    "name": "JOT_Date_Transformed2",
    "nCols": 40,
    "nRows": 18,
    "nCells": 720,
    "nCellsReconciliated": 0,
    "lastModifiedDate": "2024-05-06T09:34:53.131Z"
  },
  "columns": {
    "Fecha_id": {
      "id": "Fecha_id",
      "label": "Fecha_id",
      "status": "empty",
      "context": {},
      "metadata": []
    },
    "Cuenta_id": {
      "id": "Cuenta_id",
      "label": "Cuenta_id",
      "status": "empty",
      "context": {},
      "metadata": []
    },
    "Campa\u00f1a_id": {
      "id": "Campa\u00f1a_id",
      "label": "Campa\u00f1a_id",
      "status": "empty",
      "context": {},
      "metadata": []
    },
    "Grupo_id": {
      "id": "Grupo_id",
      "label": "Grupo_id",
      "status": "empty",
      "context": {},
      "metadata": []
    },
    "Keyword_id": {
      "id": "Keyword_id",
      "label": "Keyword_id",
      "status": "empty",
      "context": {},
      "metadata": []
    }

In [16]:
import requests
import pandas as pd
import json

SEMTUI_URI = "http://localhost:3003/api"  # Make sure this points to your actual API

def getReconciliatorData():
    """
    Retrieves reconciliator data from the backend

    :return: data of reconciliator services in JSON format
    """
    response = requests.get(SEMTUI_URI + '/reconciliators/list')
    return json.loads(response.text)

def cleanServiceList(serviceList):
    """
    Cleans and formats the service list

    :serviceList: data regarding available services
    :return: dataframe containing reconciliators information
    """
    serviceList = serviceList
    reconciliators = pd.DataFrame(columns=["id", "relativeUrl", "name"])
    for reconciliator in serviceList:
        reconciliators.loc[len(reconciliators)] = [
            reconciliator["id"], reconciliator["relativeUrl"], reconciliator["name"]]
    return reconciliators

def getReconciliatorsList():
    """
    Provides a list of available reconciliators with their main information

    :return: a dataframe containing reconciliators and their information
    """
    response = getReconciliatorData()
    return cleanServiceList(response)



def getExtenderData():
    """
    Retrieves extender data from the backend

    :return: data of extension services in JSON format
    """
    response = requests.get(SEMTUI_URI + '/extenders/list')
    return json.loads(response.text)

def getExtendersList():
    """
    Provides a list of available extenders with their main information

    :return: a dataframe containing extenders and their information
    """
    response = getExtenderData()
    return cleanServiceList(response)


In [17]:
getReconciliatorsList()

,id,relativeUrl,name
0,hereGeoCoords,/here,Here Geo Location
1,wikidata,/wikidata,Wikidata


In [18]:
getExtendersList()

,id,relativeUrl,name
0,columnExt,,Column Extender
1,hereRoute,,HERE Route
2,openMeteoExtender,,Open Meteo Properties
3,parametricWikiDataSPARQL,/wikidata/entities,Parametric Wikidata SPARQL
4,parametricWikiDataSelectSPARQL,/wikidata/entities,Wikidata Select SPARQL
5,wikidataGeoProperties,/wikidata/entities,Wikidata Geo Properties
6,wikidataGeoPropertiesSPARQL,/wikidata/entities,Wikidata Geo Properties SPARQL


In [26]:
SEMTUI_URI = "http://localhost:3003/api/"  # Ensure this points to your actual API endpoint

def getReconciliatorData():
    """
    Retrieves reconciliator data from the backend
    :return: data of reconciliator services in JSON format
    """
    try:
        response = requests.get(SEMTUI_URI + '/reconciliators/list')
        response.raise_for_status()  # Raise an exception for 4xx or 5xx status codes
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error occurred while retrieving reconciliator data: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"Error occurred while parsing reconciliator data: {e}")
        print(f"Response content: {response.text}")
        return None

def getReconciliator(idReconciliator, response):
    """
    Function that, given the reconciliator's ID, returns a dictionary 
    with all the service information

    :idReconciliator: the ID of the reconciliator in question
    :return: a dictionary with the reconciliator's information
    """
    for reconciliator in response:
        if reconciliator['id'] == idReconciliator:
            return {
                'uri': reconciliator['uri'],
                'prefix': reconciliator['prefix'],
                'name': reconciliator['name'],
                'relativeUrl': reconciliator['relativeUrl']
            }
    return None

def createReconciliationPayload(table, columnName, idReconciliator):
    """
    Creates the payload for the reconciliation request

    :table: table in raw format
    :columnName: the name of the column to reconcile
    :idReconciliator: the id of the reconciliation service to use
    :return: the request payload
    """
    rows = []
    rows.append({"id": 'column$index', "label": columnName})
    for row in table['rows'].keys():
        rows.append({"id": row+"$"+columnName,
                    "label": table['rows'][row]['cells'][columnName]['label']})
    return {"serviceId": idReconciliator, "items": rows}

def parseNameField(name, uriReconciliator, idEntity):
    """
    The actual function that changes the name format to the one required for visualization

    :name: entity name
    :uriReconciliator: the URI of the affiliated knowledge graph
    :idEntity: entity ID
    :return: the name in the correct format
    """
    return {
        'value': name,
        'uri': uriReconciliator + idEntity
    }

def createCellMetadataNameField(metadata, idReconciliator, reconciliatorResponse):
    """
    Refactor of the name field within cell-level metadata
    necessary for visualization within SEMTUI

    :metadata: column-level metadata
    :idReconciliator: ID of the reconciliator performed in the operation
    :reconciliatorResponse: response containing reconciliator information
    :return: metadata containing the name field in the new format
    """
    for row in range(len(metadata)):
        try:
            for item in range(len(metadata[row]["metadata"])):
                value = metadata[row]["metadata"][item]['name']
                uri = metadata[row]["metadata"][item]['id']
                metadata[row]["metadata"][item]['name'] = parseNameField(value, getReconciliator(
                    idReconciliator, reconciliatorResponse)['uri'], uri.split(':')[1])
        except:
            return []
    return metadata

def calculateScoreBoundCell(metadata):
    """
    Calculates the min and max value of the score of the results obtained for
    a single cell

    :metadata: metadata of a single cell
    :return: a dictionary containing the two values
    """
    try:
        scoreList = [item['score'] for item in metadata]
        return {'lowestScore': min(scoreList), 'highestScore': max(scoreList)}
    except:
        return {'lowestScore': 0, 'highestScore': 0}

def valueMatchCell(metadata):
    """
    Returns whether a cell has obtained a match or not

    :metadata: cell-level metadata
    :return: True or False based on the match occurrence
    """
    for item in metadata:
        if item['match'] == True:
            return True
    return False

def createAnnotationMetaCell(metadata):
    """
    Creates the annotationMeta field at the cell level, 
    which will then be inserted into the table

    :metadata: cell-level metadata
    :return: the dictionary with data regarding annotationMeta
    """
    scoreBound = calculateScoreBoundCell(metadata)
    return {'annotated': True,
            'match': {'value': valueMatchCell(metadata)},
            'lowestScore': scoreBound['lowestScore'],
            'highestScore': scoreBound['highestScore']}


def updateMetadataCells(table, metadata):
    """
    Allows inserting new cell-level metadata

    :table: table in raw format
    :metadata: cell-level metadata
    :return: the table in raw format with metadata
    """
    for item in metadata:
        item["id"] = item["id"].split("$")
        try:
            table["rows"][item["id"][0]]["cells"][item["id"]
                                                  [1]]["metadata"] = item["metadata"]
            table["rows"][item["id"][0]]["cells"][item["id"][1]
                                                  ]["annotationMeta"] = createAnnotationMetaCell(item["metadata"])
        except:
            print("")
    return table

def calculateNCellsReconciliatedColumn(table, columnName):
    """
    Calculates the number of reconciled cells within 
    a column

    :table: table in raw format
    :columnName: name of the column in question
    :return: the number of reconciled cells
    """
    cellsReconciliated = 0
    rowsIndex = table["rows"].keys()
    for row in rowsIndex:
        try:
            if table['rows'][row]['cells'][columnName]['annotationMeta']["annotated"] == True:
                cellsReconciliated += 1
        except:
            cellsReconciliated = cellsReconciliated
    return cellsReconciliated


def createContextColumn(table, columnName, idReconciliator, reconciliatorResponse):
    """
    Creates the context field at the column level by retrieving the necessary data

    :table: table in raw format
    :columnName: the name of the column for which the context is being created
    :idReconciliator: the ID of the reconciliator used for the column
    :reconciliatorResponse: response containing reconciliator information
    :return: the context field of the column
    """
    nCells = len(table["rows"].keys())
    reconciliator = getReconciliator(idReconciliator, reconciliatorResponse)
    return {reconciliator['prefix']: {
            'uri': reconciliator['uri'],
            'total': nCells,
            'reconciliated': calculateNCellsReconciliatedColumn(table, columnName)
            }}

def getColumnMetadata(metadata):
    """
    Allows retrieving column-level data, particularly
    the entity corresponding to the column, the column types,
    and the match value of the entities in the column

    :metadata: column metadata obtained from the reconciliator
    :return: dictionary containing the different data
    """
    entity = []
    types = []
    for i in range(len(metadata)):
        try:
            if metadata[i]['id'] == ['column', 'index']:
                entity = metadata[i]['metadata']
        except:
            print("No column entity is provided")
        try:
            if metadata[i]['id'] != ['column', 'index']:
                for j in range(len(metadata[i]['metadata'])):
                    if metadata[i]['metadata'][j]['match'] == True:
                        types.append(metadata[i]['metadata'][j]['type'][0])
        except:
            print("No column type is provided")
    matchMetadataValue = True
    for item in entity:
        if item['match'] == False:
            matchMetadataValue = False
    return {'entity': entity, 'type': types, 'matchMetadataValue': matchMetadataValue}

def createMetadataFieldColumn(metadata):
    """
    Allows creating the metadata field for a column, which will
    then be inserted into the general column-level metadata

    :metadata: column-level metadata
    :return: the metadata field at the column level
    """
    return [
        {'id': '',
         'match': getColumnMetadata(metadata)['matchMetadataValue'],
         'score': 0,
         'name':{'value': '', 'uri': ''},
         'entity': getColumnMetadata(metadata)['entity'],
         'property':[],
         'type': getColumnMetadata(metadata)['type']}
    ]

#------------------------------------------------------------------------------
def calculateScoreBoundColumn(table, columnName, reconciliatorResponse):
    allScores = []
    matchValue = True
    rows = table["rows"].keys()
    for row in rows:
        try:
            annotationMeta = table["rows"][row]['cells'][columnName]['annotationMeta']
            if annotationMeta['annotated'] == True:
                allScores.append(annotationMeta['lowestScore'])
                allScores.append(annotationMeta['highestScore'])
            if annotationMeta['match']['value'] == False:
                matchValue = False
        except KeyError:
            print(f"Missing key in cell annotation metadata: 'annotationMeta'")
            print(f"Row: {row}, Column: {columnName}")
            print(f"Cell data: {table['rows'][row]['cells'][columnName]}")
    
    if allScores:
        return {'lowestScore': min(allScores), 'highestScore': max(allScores), 'matchValue': matchValue}
    else:
        print("No valid annotation metadata found for the column.")
        return {'lowestScore': None, 'highestScore': None, 'matchValue': None}

def createAnnotationMetaColumn(annotated, table, columnName, reconciliatorResponse):
    scoreBound = calculateScoreBoundColumn(
        table, columnName, reconciliatorResponse)
    return {'annotated': annotated,
            'match': {'value': scoreBound['matchValue']},
            'lowestScore': scoreBound['lowestScore'],
            'highestScore': scoreBound['highestScore']
            }


def updateMetadataColumn(table, columnName, idReconciliator, metadata, reconciliatorResponse):
    """
    Allows inserting column-level metadata

    :table: table in raw format
    :columnName: name of the column to operate on
    :idReconciliator: ID of the reconciliator used
    :metadata: column-level metadata
    :reconciliatorResponse: response containing reconciliator information
    :return: the table with the new metadata inserted
    """
    # inquire about the different states
    table['columns'][columnName]['status'] = 'pending'
    table['columns'][columnName]['kind'] = "entity"
    table['columns'][columnName]['context'] = createContextColumn(
        table, columnName, idReconciliator, reconciliatorResponse)
    table['columns'][columnName]['metadata'] = createMetadataFieldColumn(
        metadata)
    table['columns'][columnName]['annotationMeta'] = createAnnotationMetaColumn(
        True, table, columnName, reconciliatorResponse)
    return table

def calculateScoreBoundTable(table):
    """
    Calculates the minimum and maximum score obtained in
    the results of the entire table

    :table: the table in raw format
    :return: a dictionary containing the two values
    """
    allScores = []
    reconciliateColumns = [column for column in table['columns'].keys(
    ) if table['columns'][column]['status'] != 'empty']
    for column in reconciliateColumns:
        try:
            if table['columns'][column]['annotationMeta']['annotated'] == True:
                allScores.append(table['columns'][column]
                                 ['annotationMeta']['lowestScore'])
                allScores.append(table['columns'][column]
                                 ['annotationMeta']['highestScore'])
        except:
            print("Missed column annotation metadata")
    try:
        return {'lowestScore': min(allScores), 'highestScore': max(allScores)}
    except:
        return {'lowestScore': 0, 'highestScore': 0}

def calculateNCellsReconciliated(table):
    """
    Calculates the number of reconciled cells within the
    table

    :table: the table in raw format
    :return: the number of reconciled cells
    """
    cellsReconciliated = 0
    columnsName = table['columns'].keys()
    for column in columnsName:
        try:
            contextReconciliator = table['columns'][column]['context'].keys()
            for reconcliator in contextReconciliator:
                cellsReconciliated += int(table['columns'][column]
                                          ['context'][reconcliator]['reconciliated'])
        except:
            cellsReconciliated += 0
    return cellsReconciliated


def updateMetadataTable(table):
    """
    Inserts metadata at the table level

    :table: table in raw format
    :return: the table with the new metadata inserted
    """
    scoreBound = calculateScoreBoundTable(table)
    table['table']['minMetaScore'] = scoreBound['lowestScore']
    table['table']['maxMetaScore'] = scoreBound['highestScore']
    table['table']['nCellsReconciliated'] = calculateNCellsReconciliated(table)
    return table

def reconcile(table, columnName, idReconciliator):
    """
    Reconciles a column with the chosen reconciliator

    :table: the table with the column to reconcile 
    :columnName: the name of the column to reconcile 
    :idReconciliator: ID of the reconciliator to use 
    :return: table with reconciled column
    """
    reconciliatorResponse = getReconciliatorData()
    reconciliator = getReconciliator(idReconciliator, reconciliatorResponse)
    if reconciliator is None:
        print(f"Reconciliator with ID {idReconciliator} not found.")
        return None
    # creating the request
    url = SEMTUI_URI + '/reconciliators' + str(reconciliator['relativeUrl'])
    payload = createReconciliationPayload(table, columnName, idReconciliator)
    response = requests.post(url, json=payload)
    response = json.loads(response.text)
    # inserting data into the table
    metadata = createCellMetadataNameField(response, idReconciliator, reconciliatorResponse)
    table = updateMetadataCells(table, metadata)
    table = updateMetadataColumn(table, columnName, idReconciliator, metadata, reconciliatorResponse)
    table = updateMetadataTable(table)
    return {'raw': table}

In [27]:
table = table_json
columnName = "City"
idReconciliator = "hereGeoCoords"
Reconciled_data = reconcile(table, columnName, idReconciliator)

In [28]:
Reconciled_data

{'raw': {'table': {'id': '44',
   'idDataset': '13',
   'name': 'JOT_Date_Transformed2',
   'nCols': 40,
   'nRows': 18,
   'nCells': 720,
   'nCellsReconciliated': 18,
   'lastModifiedDate': '2024-05-06T09:34:53.131Z',
   'minMetaScore': 0.91,
   'maxMetaScore': 1},
  'columns': {'Fecha_id': {'id': 'Fecha_id',
    'label': 'Fecha_id',
    'status': 'empty',
    'context': {},
    'metadata': []},
   'Cuenta_id': {'id': 'Cuenta_id',
    'label': 'Cuenta_id',
    'status': 'empty',
    'context': {},
    'metadata': []},
   'Campaña_id': {'id': 'Campaña_id',
    'label': 'Campaña_id',
    'status': 'empty',
    'context': {},
    'metadata': []},
   'Grupo_id': {'id': 'Grupo_id',
    'label': 'Grupo_id',
    'status': 'empty',
    'context': {},
    'metadata': []},
   'Keyword_id': {'id': 'Keyword_id',
    'label': 'Keyword_id',
    'status': 'empty',
    'context': {},
    'metadata': []},
   'City_id': {'id': 'City_id',
    'label': 'City_id',
    'status': 'empty',
    'context': {}

In [43]:
############### 2nd method reconciliation #################

import requests
from requests.exceptions import RequestException


def getExtender(idExtender, response):
    """
    Given the extender's ID, returns the main information in JSON format

    :idExtender: the ID of the extender in question
    :response: JSON containing information about the extenders
    :return: JSON containing the main information of the extender
    """
    for extender in response:
        if extender['id'] == idExtender:
            return {
                'name': extender['name'],
                'relativeUrl': extender['relativeUrl']
            }
    return None

def createExtensionPayload(data, reconciliatedColumnName, properties, idExtender, dates=None):
    """
    Creates the payload for the extension request

    :param data: table in raw format
    :param reconciliatedColumnName: the name of the column containing reconciled id
    :param properties: the properties to use in a list format
    :param idExtender: the ID of the extender service
    :param dates: (optional) a dictionary containing date information for each row (specific to openMeteoExtender)
    :return: the request payload
    """
    items = {}
    if 'rows' not in data:
        raise KeyError("The 'data' dictionary does not contain the 'rows'")
    rows = data['rows'].keys()
    for row in rows:
        if 'cells' not in data['rows'][row]:
            raise KeyError(f"The 'data['rows'][{row}]' dictionary does not contain the 'cells' key.")
        
        cell = data['rows'][row]['cells'].get(reconciliatedColumnName)
        if cell and cell.get('annotationMeta', {}).get('match', {}).get('value') == True:
            for metadata in cell.get('metadata', []):
                if metadata.get('match') == True:
                    items[row] = metadata.get('id')
                    break    
    payload = {
        "serviceId": idExtender,
        "items": {
            str(reconciliatedColumnName): items
        },
        "property": properties
    }
    
    if idExtender == "openMeteoExtender":
        if dates is None:
            raise ValueError("The 'dates' parameter is required for the openMeteoExtender.")
        payload["dates"] = dates
        payload["weatherParams"] = properties  # This can be customized if needed
        payload["decimalFormat"] = []  # This can be customized or set externally if needed
    
    return payload

def getReconciliatorFromPrefix(prefixReconciliator, response):
    """
    Function that, given the reconciliator's prefix, returns a dictionary 
    with all the service information

    :prefixReconciliator: the prefix of the reconciliator in question
    :return: a dictionary with the reconciliator's information
    """
    for reconciliator in response:
        if reconciliator['prefix'] == prefixReconciliator:
            return {
                'uri': reconciliator['uri'],
                'id': reconciliator['id'],
                'name': reconciliator['name'],
                'relativeUrl': reconciliator['relativeUrl']
            }
    return None

def getColumnIdReconciliator(table, columnName, reconciliatorResponse):
    """
    Specifying the column of interest returns the reconciliator's ID,
    if the column is reconciled

    :table: table in raw format
    :columnName: name of the column in question
    :return: the ID of the reconciliator used
    """
    prefix = list(table['columns'][columnName]['context'].keys())
    return getReconciliatorFromPrefix(prefix[0], reconciliatorResponse)['id']

def checkEntity(newColumnData):
    entity = False
    # Assuming newColumnData refers to a row payload, which contains a 'cells' list
    for cell in newColumnData['cells']:
        # Check if there is metadata and it's not empty
        if 'metadata' in cell and cell['metadata']:
            entity = True
            break
    return entity


def parseNameEntities(entities, uriReconciliator):
    """
    Function iterated in parseNameMetadata, works at the entity level

    :param entities: List of entities present in the cell/column
    :param uriReconciliator: the URI of the affiliated knowledge graph
    :return: List of entities with updated names
    """
    for entity in entities:
        if 'id' in entity and ':' in entity['id']:
            entity_type = entity['id'].split(':')[1]  # Safely extract after colon
            entity['name'] = parseNameField(
                entity.get('name', ''),  # Safely access 'name'
                uriReconciliator,
                entity_type
            )
    return entities

def addExtendedCell(table, newColumnData, newColumnName, idReconciliator, reconciliatorResponse):
    if 'cells' not in newColumnData:
        raise ValueError("newColumnData must contain 'cells'")
    
    rowKeys = newColumnData['cells']
    entity = checkEntity(newColumnData)
    columnType = newColumnData.get('kind', 'entity' if entity else 'literal')

    for rowKey in rowKeys:
        cellData = newColumnData['cells'][rowKey]
        newCell = table['rows'][rowKey]['cells'].setdefault(newColumnName, {})
        newCell['id'] = f"{rowKey}${newColumnName}"
        newCell['label'] = cellData.get('label', '')

        uriReconciliator = getReconciliator(idReconciliator, reconciliatorResponse)['uri']
        newCell['metadata'] = parseNameEntities(cellData.get('metadata', []), uriReconciliator)

        if columnType == 'entity':
            newCell['annotationMeta'] = createAnnotationMetaCell(newCell['metadata'])
        else:
            newCell['annotationMeta'] = {}

    return table

def addExtendedColumns(table, extensionData, newColumnsName, reconciliatorResponse):
    """
    Allows iterating the operations to insert a single column for
    all the properties to be inserted.

    :param table: table in raw format
    :param extensionData: data obtained from the extender
    :param newColumnsName: names of the new columns to insert into the table
    :param reconciliatorResponse: response containing reconciliator information
    :return: the table with the new fields inserted
    """
    if 'columns' not in extensionData or 'meta' not in extensionData:
        raise ValueError("extensionData must contain 'columns' and 'meta'")

    # Iterating through new columns to be added
    for i, columnKey in enumerate(extensionData['columns'].keys()):
        if i >= len(newColumnsName):
            raise IndexError("There are more columns to add than names provided.")
        
        # Fetching reconciliator ID for the current column
        idReconciliator = getColumnIdReconciliator(
            table, extensionData['meta'][columnKey], reconciliatorResponse)
        
        # Adding the extended cell/column to the table
        table = addExtendedCell(
            table, extensionData['columns'][columnKey], newColumnsName[i], idReconciliator, reconciliatorResponse)
        
    return table

def parseNameMetadata(metadata, uriReconciliator):
    if not isinstance(metadata, list):
        raise ValueError("Expected metadata to be a list")
    
    for item in metadata:
        if 'entity' in item:
            try:
                item['entity'] = parseNameEntities(item['entity'], uriReconciliator)
            except KeyError as e:
                raise KeyError(f"Missing expected key in entity data: {str(e)}")
            except Exception as e:
                raise Exception(f"An error occurred while parsing entities: {str(e)}")
        else:
            raise KeyError("Expected 'entity' key in each metadata item")

    return metadata

def addExtendedColumns(table, extensionData, newColumnsName, reconciliatorResponse):
    if 'columns' not in extensionData or 'meta' not in extensionData:
        raise ValueError("extensionData must contain 'columns' and 'meta'")

    for i, columnKey in enumerate(extensionData['columns'].keys()):
        if i >= len(newColumnsName):
            raise IndexError("There are more columns to add than names provided.")
        
        idReconciliator = getColumnIdReconciliator(
            table, extensionData['meta'][columnKey], reconciliatorResponse)
        
        table = addExtendedCell(
            table, extensionData['columns'][columnKey], newColumnsName[i], idReconciliator, reconciliatorResponse)
        
    return table

def extendColumn(table, reconciliatedColumnName, idExtender, properties, newColumnsName, dateColumnName):
    """
    Allows extending specified properties present in the Knowledge Graph as a new column.

    :param table: the table containing the data
    :param reconciliatedColumnName: the column containing the ID in the KG
    :param idExtender: the extender to use for extension
    :param properties: the properties to extend in the table
    :param newColumnsName: the name of the new columns to add
    :param dateColumnName: the name of the date column to extract date information for each row
    :return: the extended table
    """
    # Prepare the dates information dynamically
    dates = {}
    for row_key, row_data in table['rows'].items():
        # Safely extract date_value from the label key of the date column
        date_value = row_data['cells'].get(dateColumnName, {}).get('label')
        
        if date_value:
            dates[row_key] = [date_value]
        else:
            print(f"Missing or invalid date for row {row_key}, skipping this row.")
            continue  # Optionally skip this row or handle accordingly

    reconciliatorResponse = getReconciliatorData()
    extenderData = getExtender(idExtender, getExtenderData())
    url = SEMTUI_URI + "extenders/" + extenderData['relativeUrl']
    payload = createExtensionPayload(table, reconciliatedColumnName, properties, idExtender, dates)
    headers = {"Accept": "application/json"}
    try:
        response = requests.post(url, json=payload, headers=headers)
        print(f"HTTP Status Code: {response.status_code}")
        print(f"HTTP Response Text: '{response.text}'")
        response.raise_for_status()
        data = response.json()
        table = addExtendedColumns(table, data, newColumnsName, reconciliatorResponse)
        return table
    except requests.RequestException as e:
        print(f"Network-related error occurred: {e}")
    except json.JSONDecodeError as e:
        print("Error parsing JSON response:", e)
        print("Raw response for debugging:", response.text)
    except KeyError as e:
        print(f"Missing expected key in data:", e)


In [44]:
# Usage example assuming other function dependencies are properly defined and available
update_payload = Reconciled_data  # Assuming 'result' is defined elsewhere
reconciliatedColumnName = "City"
idExtender = "openMeteoExtender"  # ID for Open Meteo Properties extender
properties = ["apparent_temperature_max", "apparent_temperature_min", "precipitation_sum"]
newColumnsName = ["apparent_temperature_max", "apparent_temperature_min", "precipitation_sum"]
dateColumnName = "Fecha_id"  # Column name for the date

New_data = extendColumn(update_payload['raw'], reconciliatedColumnName, idExtender, properties, newColumnsName, dateColumnName)


HTTP Status Code: 200
HTTP Response Text: '{"columns":{"City_apparent_temperature_max":{"label":"City_apparent_temperature_max","metadata":[],"cells":{"r0":{"label":"29,9","metadata":[]},"r1":{"label":"30,7","metadata":[]},"r2":{"label":"32,4","metadata":[]},"r3":{"label":"25","metadata":[]},"r4":{"label":"34,9","metadata":[]},"r5":{"label":"32,2","metadata":[]},"r6":{"label":"40,1","metadata":[]},"r7":{"label":"36,5","metadata":[]},"r8":{"label":"27,5","metadata":[]},"r9":{"label":"42,2","metadata":[]},"r10":{"label":"41,6","metadata":[]},"r11":{"label":"41,4","metadata":[]},"r12":{"label":"23,9","metadata":[]},"r13":{"label":"31,9","metadata":[]},"r14":{"label":"17,7","metadata":[]},"r15":{"label":"37","metadata":[]},"r16":{"label":"36,1","metadata":[]},"r17":{"label":"30,2","metadata":[]}}},"City_apparent_temperature_min":{"label":"City_apparent_temperature_min","metadata":[],"cells":{"r0":{"label":"20,3","metadata":[]},"r1":{"label":"22,4","metadata":[]},"r2":{"label":"22,8","metad

In [46]:
New_data

{'table': {'id': '44',
  'idDataset': '13',
  'name': 'JOT_Date_Transformed2',
  'nCols': 40,
  'nRows': 18,
  'nCells': 720,
  'nCellsReconciliated': 18,
  'lastModifiedDate': '2024-05-06T09:34:53.131Z',
  'minMetaScore': 0.91,
  'maxMetaScore': 1},
 'columns': {'Fecha_id': {'id': 'Fecha_id',
   'label': 'Fecha_id',
   'status': 'empty',
   'context': {},
   'metadata': []},
  'Cuenta_id': {'id': 'Cuenta_id',
   'label': 'Cuenta_id',
   'status': 'empty',
   'context': {},
   'metadata': []},
  'Campaña_id': {'id': 'Campaña_id',
   'label': 'Campaña_id',
   'status': 'empty',
   'context': {},
   'metadata': []},
  'Grupo_id': {'id': 'Grupo_id',
   'label': 'Grupo_id',
   'status': 'empty',
   'context': {},
   'metadata': []},
  'Keyword_id': {'id': 'Keyword_id',
   'label': 'Keyword_id',
   'status': 'empty',
   'context': {},
   'metadata': []},
  'City_id': {'id': 'City_id',
   'label': 'City_id',
   'status': 'empty',
   'context': {},
   'metadata': []},
  'State_id': {'id': 'St

In [49]:
import json
import pandas as pd

# Load the JSON data from file
data = New_data

# Assuming 'columns' and 'rows' keys are correctly located
columns = data['columns']
rows = data['rows']

# Creating a list of dictionaries for each row
data_list = []
for row_id, row_info in rows.items():
    row_data = {}
    # Include all cell keys from the row, not just those listed in the columns dictionary
    for cell_key, cell_value in row_info['cells'].items():
        row_data[cell_key] = cell_value['label']
    data_list.append(row_data)

# Convert list to DataFrame
df = pd.DataFrame(data_list)

# Display the DataFrame
print(df)

      Fecha_id   Cuenta_id   Campaña_id        Grupo_id       Keyword_id  City_id State_id Country_id                  Keyword Impresiones Clicks      Cpc QualityScore                 City          County        Country Impresiones_q1 Impresiones_q3 Impresiones_med Impresiones_iqr Impresiones_max Impresiones_min Clicks_q1 Clicks_q3 Clicks_med Clicks_iqr Clicks_max Clicks_min  Cpc_q1  Cpc_q3 Cpc_med Cpc_iqr    Cpc_max Cpc_min Qs_q1 Qs_q3 Qs_med Qs_iqr Qs_max Qs_min apparent_temperature_max apparent_temperature_min precipitation_sum
0   2023-07-01  1018571837  17309187968  138000000000.0   299000000000.0  1023619    21168       2840  5th third bank cd rates           4      5  9000000                           Chardon            Ohio  United States              1              1               1               0            2794               0         0         0          0          0        148          0  160000  620000  300000  460000  999000000   10000     0     0      0      0     10  

In [47]:
def create_update_payload(table_json):
    """
    Creates the payload required to perform the table update operation

    :param table_json: JSON representation of the table
    :return: request payload
    """
    payload = {
        "tableInstance": {
            "id": table_json["table"]["id"],
            "idDataset": table_json["table"]["idDataset"],
            "name": table_json["table"]["name"],
            "nCols": table_json["table"]["nCols"],
            "nRows": table_json["table"]["nRows"],
            "nCells": table_json["table"]["nCells"],
            "nCellsReconciliated": table_json["table"]["nCellsReconciliated"],
            "lastModifiedDate": table_json["table"]["lastModifiedDate"],
            "minMetaScore": table_json["table"]["minMetaScore"],
            "maxMetaScore": table_json["table"]["maxMetaScore"]
        },
        "columns": {
            "byId": {},
            "allIds": []
        },
        "rows": {
            "byId": {},
            "allIds": []
        }
    }

    # Process columns
    for column_id, column_data in table_json["columns"].items():
        payload["columns"]["byId"][column_id] = {
            "id": column_data["id"],
            "label": column_data["label"],
            "status": column_data["status"],
            "context": column_data["context"],
            "metadata": column_data["metadata"],
            "annotationMeta": column_data.get("annotationMeta", {})
        }
        payload["columns"]["allIds"].append(column_id)

    # Process rows
    for row_id, row_data in table_json["rows"].items():
        payload["rows"]["byId"][row_id] = {
            "id": row_data["id"],
            "cells": {}
        }
        for cell_id, cell_data in row_data["cells"].items():
            payload["rows"]["byId"][row_id]["cells"][cell_id] = {
                "id": cell_data["id"],
                "label": cell_data["label"],
                "metadata": cell_data["metadata"],
                "annotationMeta": cell_data.get("annotationMeta", {})
            }
        payload["rows"]["allIds"].append(row_id)

    return payload

In [48]:
# API configuration
API_URL = "http://localhost:3003/api/"
DATASETS_ENDPOINT = "dataset/"
dataset_id = "13"
table_id = "44"

token = token_manager.get_token()
headers = {
    'Authorization': f'Bearer {token}',
    'Content-Type': 'application/json'
}

url = f"{API_URL}{DATASETS_ENDPOINT}{dataset_id}/table/{table_id}"

# Create the update payload
update_payload = create_update_payload(New_data)

try:
    # Send the PUT request to update the table
    response = requests.put(url, headers=headers, json=update_payload)
    
    if response.status_code == 200:
        print("Table updated successfully!")
        response_data = response.json()
        print("Response data:", response_data)
    elif response.status_code == 401:
        print("Unauthorized: Invalid or missing token.")
    elif response.status_code == 404:
        print(f"Dataset or table with ID {dataset_id}/{table_id} not found.")
    else:
        print(f"Failed to update table: {response.status_code}, {response.text}")
except requests.exceptions.RequestException as e:
    print(f"Error occurred while updating table: {e}")

Table updated successfully!
Response data: {'id': '44', 'idDataset': '13', 'name': 'JOT_Date_Transformed2', 'nCols': 40, 'nRows': 18, 'nCells': 720, 'nCellsReconciliated': 18, 'lastModifiedDate': '2024-05-06T10:54:15.115Z', 'minMetaScore': 0.91, 'maxMetaScore': 1}


In [50]:
# Example usage
dataset_id = "13"
table_name = "JOT_Date_Transformed2"  # Replace with the desired table name

table_json = get_table_by_name(dataset_id, table_name, token_manager)

if table_json is not None:
    # Process the table data as needed
    print("Table JSON:")
    print(json.dumps(table_json, indent=2))

Table JSON:
{
  "table": {
    "id": "44",
    "idDataset": "13",
    "name": "JOT_Date_Transformed2",
    "nCols": 40,
    "nRows": 18,
    "nCells": 720,
    "nCellsReconciliated": 18,
    "lastModifiedDate": "2024-05-06T10:54:15.115Z",
    "minMetaScore": 0.91,
    "maxMetaScore": 1
  },
  "columns": {
    "Fecha_id": {
      "id": "Fecha_id",
      "label": "Fecha_id",
      "status": "empty",
      "context": {},
      "metadata": []
    },
    "Cuenta_id": {
      "id": "Cuenta_id",
      "label": "Cuenta_id",
      "status": "empty",
      "context": {},
      "metadata": []
    },
    "Campa\u00f1a_id": {
      "id": "Campa\u00f1a_id",
      "label": "Campa\u00f1a_id",
      "status": "empty",
      "context": {},
      "metadata": []
    },
    "Grupo_id": {
      "id": "Grupo_id",
      "label": "Grupo_id",
      "status": "empty",
      "context": {},
      "metadata": []
    },
    "Keyword_id": {
      "id": "Keyword_id",
      "label": "Keyword_id",
      "status": "empty

In [40]:
from yarl import URL
url = URL('https://www.python.org/~guido?arg=1#frag')
url

URL('https://www.python.org/~guido?arg=1#frag')

In [41]:
SEMTUI_URI = URL(SEMTUI_URI)

In [42]:
testuri / 'extender'

URL('http://localhost:3003/api/extender')

In [35]:
res = requests.get(sempath)

InvalidURL: Invalid URL 'http:/localhost:3003/api': No host supplied